# Network Flow Problems

In [1]:
import numpy as np
from sympy import *
from scipy.optimize import linprog
import warnings
warnings.filterwarnings('ignore')

# Problem 1

### Problem Statement

Your company will produce three new products. At this moment, 5 of your plants have excess production capacity. The respective unit manufacturing costs for the first product will be $30, $29, $32, $28, and $29 in plants 1, 2, 3, 4, and 5. The unit manufacturing cost of the second product will be $45, $41, $46, $42, and $43 in plants 1, 2, 3, 4, and 5, respectively, and for the third product, it will be $38, $35, and $40 in plants 1, 2, and 3, but plants 4 and 5 cannot manufacture this third product.

The sales department indicates to you that the daily production should be 1400, 1000, and 600 units of products 1, 2, and 3, respectively. Plants 1, 2, 3, 4, and 5 have capacity to produce 400, 600, 400, 600, and 1000 units daily, regardless of the product or combination of products.

You want to assign the new products to the plants in such a way that you obtain the minimum total manufacturing cost.


## a) Summary

| Plant  |  |  Cost/Product |   | 
|:-:|:-----------:|:-----:|:-----:|
|   |      1      |   2   |   3   |
| 1 |    \$30     | \$45  | \$38  | 
| 2 |    \$29     | \$41  | \$35  | 
| 3 |    \$32     | \$46  | \$40  | 
| 4 |    \$28     | \$42  | N/A   | 
| 5 |    \$29     | \$43  | N/A   | 

Total plant capacities = 3000 

Total product productions = 3000


## b) Problem Formulation

The mathematical problem to be solved is a minimum-cost flow problem. Specifically, it is a transportation problem. The decision variables of this problem are the quantities of new products to be produced. 

In this regard, we will use the notation $x_{ij}$ to represent product $j$, produced in plant $i$. The products $x_{ij}$ are the decision variables of the optimization problem.

Being a minimum-cost flow problem, the objective function of the optimization problem is a function that relates the costs of transporting the products $x_{ij}$ through each arc of the network. The costs associated with each arc are related in the table from section a). Thus, the objective function is given by the following expression:

$$f(x) = 30x_{16} + 29x_{26} + 32x_{36} + 28x_{43} + 29x_{56} + 45x_{17} + 41x_{27} + 46x_{37} + 42x_{47} + 43x_{57} + 38x_{18} + 35x_{28} + 40x_{38}$$

It is important to mention that products 1, 2, and 3 were represented in the problem with numbers 6, 7, and 8, respectively, as shown in the graphical representation attached to this notebook (see the file titled "graphP1.png").

The constraints associated with this problem indicate that the difference between the amount of flow leaving node $i$ and the amount of flow entering node $i$ must be equal to a quantity $b_i$. This quantity is a supply if $b_i > 0$ or a requirement if $b_i < 0 $. In the case of this problem, these $b_i$ represent the production capacities of each plant (supplies) and the daily production of the products (requirements). Additionally, there is a constraint that the problem is balanced. That is, the sum of the requirements and supplies must be equal to zero. This condition holds for this problem, as $\sum_{i=1}^{8}b_i = 0$.

Taking into account the above and the graphical representation of the problem, the following constraints are formulated:

Plants \
$x_{16} + x_{17} + x_{18} = 400$ \
$x_{26} + x_{27} + x_{28} = 600$ \
$x_{36} + x_{37} + x_{38} = 400$ \
$x_{46} + x_{47} + x_{48} = 600$ \
$x_{56} + x_{57} + x_{58} = 1000$ 

Products \
$- (x_{16} + x_{26} + x_{36} + x_{46} + x_{56}) = -1400$ \
$- (x_{17} + x_{27} + x_{37} + x_{47} + x_{57}) = -1000$ \
$- (x_{18} + x_{28} + x_{38} + x_{48} + x_{58}) = -600$ 

Balanced problem \
$b_1 = 400$ \
$b_2 = 600$ \
$b_3 = 400$ \
$b_4 = 600$ \
$b_5 = 1000$ \
$b_6 = -1400$ \
$b_7 = -1000$ \
$b_8 = -600$

$\sum_{i=1}^{8}b_i = 0$

Finally, the optimization problem is formulated as follows: 

max $f(x) = 30x_{16} + 29x_{26} + 32x_{36} + 28x_{43} + 29x_{56} + 45x_{17} + 41x_{27} + 46x_{37} + 42x_{47} + 43x_{57} + 38x_{18} + 35x_{28} + 40x_{38}$ \
s.t.

$x_{16} + x_{17} + x_{18} = 400$ \
$x_{26} + x_{27} + x_{28} = 600$ \
$x_{36} + x_{37} + x_{38} = 400$ \
$x_{46} + x_{47} + x_{48} = 600$ \
$x_{56} + x_{57} + x_{58} = 1000$ \
$- (x_{16} + x_{26} + x_{36} + x_{46} + x_{56}) = -1400$ \
$- (x_{17} + x_{27} + x_{37} + x_{47} + x_{57}) = -1000$ \
$- (x_{18} + x_{28} + x_{38} + x_{48} + x_{58}) = -600$


## d) Problem solution

In [2]:
f = np.array([30,29,32,28,29,45,41,46,42,43,38,35,40,0,0])
A = np.array([[1,0,0,0,0,1,0,0,0,0,1,0,0,0,0],
             [0,1,0,0,0,0,1,0,0,0,0,1,0,0,0],
             [0,0,1,0,0,0,0,1,0,0,0,0,1,0,0],
             [0,0,0,1,0,0,0,0,1,0,0,0,0,0,0],
             [0,0,0,0,1,0,0,0,0,1,0,0,0,0,0],
             [-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,-1,-1,-1,-1,-1,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1]])
b = np.array([[400],[600],[400],[600],[1000],[-1400],[-1000],[-600]])

bounds = []

for i in range(15):
    bounds.insert(i,tuple((0,None)))

res = linprog(f, A_ub=None, b_ub=None, A_eq = A, b_eq = b, bounds = bounds, method='simplex', options={"disp": True})
print(res)

Optimization terminated successfully.
         Current function value: 105600.000000
         Iterations: 12
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 105600.0
       x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
     nit: 12


The manufacturing cost is $105,600. The production by plants is as follows:
- Plant 1: Does not produce any products
- Plant 2: Produces 400 units of product 1 and 1000 units of product 2
- Plant 3: Produces 400 units of product 1, 400 units of product 2, and 200 units of product 3
- Plant 4: Produces 400 units of product 2 and 200 units of product 3
- Plant 5: Does not produce any products

# Problem 2

## Problem Statement

Consider the problem of evacuating people from a public building (such as a school or a university). For example, consider the floor plan with five classrooms (A, B, C, D, E) and a hallway (F) depicted in the "building.png" file attached to this notebook. Each room will be connected to other room(s) via corridors, and each corridor has a limited flow of people per minute. There are three different types of corridors in the building. The building has three exits (located in the Hall and in classrooms C and E).

A graphical representation of the network is found in the "graphP2b.png" file, considering that the problem to be solved involves finding the maximum number of people that can be evacuated from the building in a given time. Note: a source node *s* (a node that feeds people into each classroom and the hall) and a sink node *t* (a node where all people converge - meeting point) are added.


## b) Problem Formulation to evacuate the max number of people in 1 minute

Below is the problem statement to maximize the maximum number of people that can be evacuated from the building in 1 minute. The maximum capacities of people in each classroom (node) of the building are detailed below:

1. Classroom A: 20 people 

2. Classroom B: 25 people

3. Classroom C: 50 people

4. Classroom D: 30 people

5. Classroom E: 20 people

6. Hall F: 80 people

Additionally, a source node named s was created that inserts people into the classrooms, and a sink node that serves as the evacuation meeting point where all people arrive after leaving the building. This node was named t. For a better organization of the problem, a number was assigned to each node, and it was defined that the flow between two nodes $i$ and $j$ is $x_{ij}$. These flows are the decision variables of the optimization problem.

The maximum flow problem aims to maximize the flow of the network. This means that it intends to maximize either the amount of flow leaving the source node or the amount of flow entering the sink node. Following the nomenclature described above and that can be seen in the image "graphP2b.png", the problem is posed as follows:

Next to each constraint, the number of the node to which the constraint belongs is noted in parentheses.

max $f(x) = x_{14} + x_{13} + x_{17} + x_{12} + x_{15} + x_{16} $ s.a.

$x_{14} + x_{13} + x_{17} + x_{12} + x_{15} + x_{16} - (x_{48} + x_{78} + x_{17} + x_{68}) = 0$ (What leaves the source node minus what enters the sink node = 0) \
$x_{27} - x_{12} = 0$ (2)\
$x_{37} - x_{13} = 0$ (3)\
$x_{48} + x_{47} - (x_{14} + x_{74}) = 0$ (4)\
$x_{57} - x_{15} = 0$ (5)\
$x_{68} + x_{67} - (x_{16} + x_{76}) = 0$ (6) \
$(x_{78} + x_{74} + x_{76}) - (x_{47} + x_{67} + x_{37} + x_{17} + x_{27} + x_{57} + x_{67}) = 0$ (7)\
$x_{47} + x_{74} \leq 50$\
$x_{67} + x_{76} \leq 15$

$0 \leq x_{14} \leq 50$ \
$0 \leq x_{13} \leq 25$ \
$0 \leq x_{17} \leq 80$ \
$0 \leq x_{12} \leq 20$ \
$0 \leq x_{15} \leq 30$ \
$0 \leq x_{16} \leq 20$ \
$0 \leq x_{48} \leq 15$ \
$0 \leq x_{78} \leq 75$ \
$0 \leq x_{68} \leq 15$ \
$0 \leq x_{37} \leq 50$ \
$0 \leq x_{27} \leq 50$ \
$0 \leq x_{57} \leq 50$ \
$0 \leq x_{47} \leq 50$ \
$0 \leq x_{67} \leq 15$

In [3]:
f = np.array([[-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0]])
A = np.array([[-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
             [0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
             [0,0,-1,0,0,0,0,0,1,1,0,0,0,-1,0,0],
             [0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0],
             [0,0,0,0,-1,0,0,0,0,0,0,1,1,0,-1,0],
             [0,0,0,0,0,-1,-1,-1,-1,0,-1,-1,0,1,1,1],
             [1,1,1,1,1,1,0,0,0,-1,0,0,-1,0,0,-1]])
b = np.zeros((7,1))

A_ub = np.array([[0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0],
                 [0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0]])

b_ub = np.array([[50], [15]])

bounds12 = (0,20)
bounds13 = (0,25)
bounds14 = (0,50)
bounds15 = (0,30)
bounds16 = (0,20)
bounds17 = (0,80)
bounds27 = (0,50)
bounds37 = (0,50)
bounds47 = (0,50)
bounds48 = (0,15)
bounds57 = (0,50)
bounds67 = (0,50)
bounds68 = (0,15)
bounds74 = (0,50)
bounds76 = (0,15)
bounds78 = (0,75)

bounds = [bounds12,bounds13,bounds14,bounds15,bounds16,bounds17,bounds27,bounds37,bounds47,bounds48,bounds57,bounds67,bounds68,bounds74,bounds76,bounds78]

res = linprog(f, A_ub=A_ub, b_ub=b_ub, A_eq = A, b_eq = b, bounds = bounds, method='simplex', options={"disp": True})
print(res)

Optimization terminated successfully.
         Current function value: -105.000000 
         Iterations: 26
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -105.0
       x: [ 2.000e+01  2.500e+01 ...  1.500e+01  7.500e+01]
     nit: 26


It is not possible to evacuate all the people in 1 minute.


## c) Problem Formulation to evacuate the max number of people in 2 minutes.

Here is the problem formulation to maximize the maximum number of people that can be evacuated from the building in 2 minutes. The maximum capacities of people in each room (node) of the building are as follows:

1. Room A: 20 people
2. Room B: 25 people
3. Room C: 50 people
4. Room D: 30 people
5. Room E: 20 people
6. Hall F: 80 people

The graph for this problem can be found in the image "graphP2c.png". The formulation is presented below:

Next to each constraint, the number of the node to which the constraint belongs is noted in parentheses.

max $f(x) = x_{14} + x_{13} + x_{17} + x_{12} + x_{15} + x_{16} $ s.t.

$x_{14} + x_{13} + x_{17} + x_{12} + x_{15} + x_{16} - (x_{48} + x_{78} + x_{17} + x_{68}) = 0$ (What leaves the source node minus what enters the sink node = 0) \
$x_{27} - x_{12} = 0$ (2)\
$x_{37} - x_{13} = 0$ (3)\
$x_{48} + x_{47} - (x_{14} + x_{74}) = 0$ (4)\
$x_{57} - x_{15} = 0$ (5)\
$x_{68} + x_{67} - (x_{16} + x_{76}) = 0$ (6) \
$(x_{78} + x_{74} + x_{76}) - (x_{47} + x_{67} + x_{37} + x_{17} + x_{27} + x_{57} + x_{67}) = 0$ (7)\
$x_{47} + x_{74} \leq 100$ \
$x_{67} + x_{76} \leq 30$

$0 \leq x_{14} \leq 50$ \
$0 \leq x_{13} \leq 25$ \
$0 \leq x_{17} \leq 80$ \
$0 \leq x_{12} \leq 20$ \
$0 \leq x_{15} \leq 30$ \
$0 \leq x_{16} \leq 20$ \
$0 \leq x_{48} \leq 30$ \
$0 \leq x_{78} \leq 150$ \
$0 \leq x_{68} \leq 30$ \
$0 \leq x_{37} \leq 100$ \
$0 \leq x_{27} \leq 100$ \
$0 \leq x_{57} \leq 100$ \
$0 \leq x_{47} \leq 100$ \
$0 \leq x_{67} \leq 30$


In [4]:
f = np.array([[-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0]])
A = np.array([[-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
             [0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
             [0,0,-1,0,0,0,0,0,1,1,0,0,0,-1,0,0],
             [0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0],
             [0,0,0,0,-1,0,0,0,0,0,0,1,1,0,-1,0],
             [0,0,0,0,0,-1,-1,-1,-1,0,-1,-1,0,1,1,1],
             [1,1,1,1,1,1,0,0,0,-1,0,0,-1,0,0,-1]])
b = np.zeros((7,1))

A_ub = np.array([[0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0],
                 [0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0]])

b_ub = np.array([[100], [30]])

bounds12 = (0,20)
bounds13 = (0,25)
bounds14 = (0,50)
bounds15 = (0,30)
bounds16 = (0,20)
bounds17 = (0,80)
bounds27 = (0,100)
bounds37 = (0,100)
bounds47 = (0,100)
bounds48 = (0,30)
bounds57 = (0,100)
bounds67 = (0,100)
bounds68 = (0,30)
bounds74 = (0,100)
bounds76 = (0,30)
bounds78 = (0,150)

bounds = [bounds12,bounds13,bounds14,bounds15,bounds16,bounds17,bounds27,bounds37,bounds47,bounds48,bounds57,bounds67,bounds68,bounds74,bounds76,bounds78]

res = linprog(f, A_ub=A_ub, b_ub=b_ub, A_eq = A, b_eq = b, bounds = bounds, method='simplex', options={"disp": True})
print(res)

Optimization terminated successfully.
         Current function value: -210.000000 
         Iterations: 24
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -210.0
       x: [ 2.000e+01  2.500e+01 ...  2.000e+01  1.500e+02]
     nit: 24


It's not possible to evacuate the building in 2 minutes, as there are 205 people exiting.

# Problem 3

## a)

Suppose the same scenario as in problem 2. But now, let's change the objective of the optimization problem. What is the minimum required time to evacuate the whole building?

From the previous problems, it is observed that there is a linear relationship between the minutes and the number of people evacuating the building. Every minute, 105 people leave the building. With this relationship, we can establish how many minutes $x$ it would take to evacuate the entire building, i.e., for 225 people to leave.

$x = 225 \text{ people} \times \frac{1 \text{ minute(s)}}{105 \text{ people}} \longrightarrow x = 2.143 \text{ minutes}$

Evacuating the entire building takes 2.143 minutes.


## b)

Suppose you can intervene in the public building by expanding two of the doors so that their flow capacities increase by 10 people per minute each. Which doors would you expand? Why?

The linear programming problem is solved by increasing doors C and F's max flow by 10 each.

In [5]:
f = np.array([[-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0]])
A = np.array([[-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
             [0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
             [0,0,-1,0,0,0,0,0,1,1,0,0,0,-1,0,0],
             [0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0],
             [0,0,0,0,-1,0,0,0,0,0,0,1,1,0,-1,0],
             [0,0,0,0,0,-1,-1,-1,-1,0,-1,-1,0,1,1,1],
             [1,1,1,1,1,1,0,0,0,-1,0,0,-1,0,0,-1]])
b = np.zeros((7,1))

A_ub = np.array([[0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0],
                 [0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0]])

b_ub = np.array([[50], [15]])

bounds12 = (0,20)
bounds13 = (0,25)
bounds14 = (0,50)
bounds15 = (0,30)
bounds16 = (0,20)
bounds17 = (0,80)
bounds27 = (0,50)
bounds37 = (0,50)
bounds47 = (0,50)
bounds57 = (0,50)
bounds67 = (0,50)
bounds74 = (0,50)
bounds76 = (0,15)

bounds48 = (0,25)
bounds68 = (0,15)
bounds78 = (0,85)

bounds1 = [bounds12,bounds13,bounds14,bounds15,bounds16,bounds17,bounds27,bounds37,bounds47,bounds48,bounds57,bounds67,bounds68,bounds74,bounds76,bounds78]

res1 = linprog(f, A_ub=A_ub, b_ub=b_ub, A_eq = A, b_eq = b, bounds = bounds1, method='simplex', options={"disp": True})
print(res1)

Optimization terminated successfully.
         Current function value: -125.000000 
         Iterations: 26
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -125.0
       x: [ 2.000e+01  2.500e+01 ...  1.500e+01  8.500e+01]
     nit: 26


In [6]:
bounds48 = (0,25)
bounds68 = (0,15)
bounds78 = (0,85)

bounds1 = [bounds12,bounds13,bounds14,bounds15,bounds16,bounds17,bounds27,bounds37,bounds47,bounds48,bounds57,bounds67,bounds68,bounds74,bounds76,bounds78]

res1 = linprog(f, A_ub=A_ub, b_ub=b_ub, A_eq = A, b_eq = b, bounds = bounds1, method='simplex', options={"disp": True})
print(res1)

Optimization terminated successfully.
         Current function value: -125.000000 
         Iterations: 26
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -125.0
       x: [ 2.000e+01  2.500e+01 ...  1.500e+01  8.500e+01]
     nit: 26


It is decided to expand doors C and F since they provide the highest number of evacuees in one minute, with 125 people.

## c)

With this expansion, what is the minimum required time to evacuate the building?

With the new linear relationship between minutes and the number of people evacuating the building—125 people per minute—we can determine how many minutes it would take to evacuate the entire building, i.e., for 225 people to leave.

$x = 225 \text{ people} * \frac{1 \text{ minute(s)}}{125 \text{ people}} \longrightarrow x = 1.8 \text{ minutes}$ 

Evacuating the entire building takes 1.8 minutes after expanding 2 doors.
